### アノテーション用  

In [ ]:
# coding:utf-8
import json
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import os
import glob
import seaborn as sns
import pickle
import math

In [ ]:
# 解析基準位置に移動する関数
def move_root_dir(root_dir):
    while not (os.path.basename(os.getcwd())== root_dir):
        os.chdir('../')
    print('moving to:',os.path.basename(os.getcwd()))
    return os.getcwd()

# 関数ボックス
def read_csv_file(csv_file):
    df = pd.read_csv(csv_file)
    return df

def plot_confidence_fix(df_data, title, joint_list,head):
    # confidenc
    #title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_head_confidence'
    cols=[]
    offsets=[]
    colors=[]
    
    #joint_list=[29,28,27,30,31]
    joint_name_list = [
            ['pelvis', 0],
            ['naval', 1],
            ['chest', 2],
            ['neck', 3],
            ['l_clavicle', 4],
            ['l_shoulder', 5],
            ['l_elbow', 6],
            ['l_wrist', 7],
            ['l_hand', 8],
            ['l_handtip', 9],
            ['l_thumb', 10],
            ['r_clavicle', 11],
            ['r_shoulder', 12],
            ['r_elbow', 13],
            ['r_wrist', 14],
            ['r_hand', 15],
            ['r_handtip', 16],
            ['r_thumb', 17],
            ['l_hip', 18],
            ['l_knee', 19],
            ['l_ankle', 20],
            ['l_foot', 21],
            ['r_hip', 22],
            ['r_knee', 23],
            ['r_ankle', 24],
            ['r_foot', 25],
            ['head', 26],
            ['nose', 27],
            ['l_eyes', 28],
            ['l_ear', 29],
            ['r_eyes', 30],
            ['r_ear', 31],]
    cm_name = 'jet'# B->G->R
    cm = plt.get_cmap(cm_name,len(joint_list))
    for ic,ii in enumerate(joint_list):
        #cols.append('data_skeletons_0_joints_{}_confidence_0'.format(ii))
        cols.append(head+joint_name_list[ii][0]+'_confidence')
        offsets.append(ic*3)
        colors.append(cm(ic))
    
    df = df_data.copy()
    plt.figure(figsize=(17,5))
    for col, color,offset,joint_no in zip(cols,colors,offsets,joint_list):
        plt.plot(df[col]+offset,label=joint_name_list[joint_no][0],color=color)
        
    plt.title(title)
    plt.legend()
    plt.grid()
    #plt.ylabel('x ')
    #plt.ylim(600,-1700)
    return plt

def get_pallet():
    cmap = plt.get_cmap("tab10")
    cmap = [cmap(0)[0:3],cmap(1)[0:3],cmap(2)[0:3],cmap(3)[0:3],cmap(4)[0:3],cmap(5)[0:3],
            cmap(6)[0:3],cmap(7)[0:3],cmap(8)[0:3],cmap(9)[0:3],cmap(10)[0:3],
            cmap(11)[0:3],cmap(12)[0:3],cmap(13)[0:3],cmap(14)[0:3],cmap(15)[0:3],
           ]
    cbox=[]
    for c in cmap:
        color='#'+format(int(c[0]*255), '02x')+format(int(c[1]*255), '02x')+format(int(c[2]*255), '02x')
        cbox.append(color)
    cmap = cbox
    return cmap

def fill_data(df_data, st, fn):
    df = df_data.copy()
    df['Timestamp'] =pd.to_datetime(df['sensing_time'])
    df = df.set_index('Timestamp')
    df = df.resample('100ms').ffill()
    df = df.reset_index(drop=False)     
    df = df[(df['Timestamp'] > pd.to_datetime(st)) & (df['Timestamp'] <= pd.to_datetime(fn)) ]
    if(len(df)<1):
        return False, df_data        
    df = df.reset_index(drop=False)
    return True, df  


In [ ]:
work_code='20201012_preprocess_data'
root_dir = '2020_moox_morikoro_project'
root_path = move_root_dir(root_dir)
input_data_dir = 'input/20201012_raw_data'
save_csv_dir = 'preprocess_data/'+work_code+'/csvfile'
if not os.path.isdir(save_csv_dir):
    os.makedirs(save_csv_dir)
save_pkl_dir= 'preprocess_data/'+work_code+'/pkl'
if not os.path.isdir(save_pkl_dir):
    os.makedirs(save_pkl_dir)
save_img_dir= 'output/20201012_output_data/fig/'+work_code
if not os.path.isdir(save_img_dir):
    os.makedirs(save_img_dir)

In [ ]:
os.chdir('src/preprocess/20201012_script')

from moox_detect_action.Detect_rule.DetectBodyStatus.detect_look_points import DetectLookPoints
look_points = DetectLookPoints(ids='2')
look_points2 = DetectLookPoints(ids='2')
look_points3 = DetectLookPoints(ids='3')
from moox_detect_action.Detect_rule.DetectBodyStatus.detect_face_direction_for_azure import DetectFaceDirection
face_direction = DetectFaceDirection()

root_path = move_root_dir(root_dir)

### データ読み込み  

In [ ]:
# # 試行リスト
# #input file name
# #input_file_names = glob.glob(input_data_dir+"/20201012_trial_list_fit.xlsx")
# input_file_names = glob.glob(input_data_dir+"/20201012_trial_list.xlsx")
# print(input_file_names)
# # xls book Open (xls, xlsxのどちらでも可能)
# input_book = pd.ExcelFile(input_file_names[0]) 
# # sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
# input_sheet_name = input_book.sheet_names
# # lenでシートの総数を確認
# num_sheet = len(input_sheet_name)
# # シートの数とシートの名前のリストの表示
# print ("Sheet の数:", num_sheet)
# print (input_sheet_name)
# input_sheet_df = input_book.parse(input_sheet_name[0], skiprows = 44)
# input_sheet_df = input_sheet_df.iloc[:,1:]
# #print(input_sheet_df)
# print('reading data')

# time_box=[]
# for i in range(len(input_sheet_df)):
#     box = [
#         input_sheet_df.at[i,'num'],
#         input_sheet_df.at[i,'p_no'],
#         input_sheet_df.at[i,'s_no'],
#         input_sheet_df.at[i,'t_no'],
#         str(input_sheet_df.at[i,'start_time_UTC']),
#         str(input_sheet_df.at[i,'finish_time_UTC']),
#         input_sheet_df.at[i,'AzureKinect'],
#         input_sheet_df.at[i,'mask'],
#         input_sheet_df.at[i,'notation'],        
#     ]
#     if(box[0]=='num'):
#         #print('skip:',box)
#         continue
#     if(box[0] <= 257):
#         #print('skip:',box)
#         continue
#     if(box[0] == 224 or box[0] == 225):
#         #print('skip:',box)
#         continue
#     time_box.append(box)
#     print(box)


In [ ]:
# # データセット作成
# exp_code = "00_20201012_exp"
# data_box=[]
# for t_box in time_box:
#     if(t_box[1]=='p99'):
#         continue
#     #if not (t_box[1]=='p28'):
#     #    continue
#     pp=t_box[1]
#     ss=t_box[2]
#     tt=t_box[3]
#     st=t_box[4]
#     fn=t_box[5]
#     kinect=t_box[6]
#     mask=t_box[7]
#     if not (tt=='t202010-92003-450' or tt=='t202010-93003-450' or tt=='t202010-92103-450' or tt=='t202010-93103-450'):
#         continue
    
#     if(ss=='s02'):
#         csv_files=glob.glob(input_data_dir+'/*/detectAction_c_2_parsed_json.csv')
#     else:
#         csv_files=glob.glob(input_data_dir+'/*/detectAction_c_3_parsed_json.csv')
#     for csv_file in csv_files:
#         df_data0 = read_csv_file(csv_file)
#         is_data,df_data1=fill_data(df_data0, st, fn)
#         if not (is_data):
#             continue
#         df_data=df_data1.copy()
#     print(st,fn, len(df_data))
#     file_code=save_csv_dir+'/data_df/part_'+pp+ss+tt+'_'+str(mask)+'.csv'
#     if not os.path.isdir(save_csv_dir+'/data_df'):
#         os.makedirs(save_csv_dir+'/data_df')
#     df_data.to_csv(file_code, index=False)
             
#     box=[pp,ss,tt,st,fn,df_data,mask]
#     data_box.append(box)
# print('data_count: ',len(data_box))
# finename = save_pkl_dir+'/'+exp_code+'.pkl'
# with open(finename, 'wb') as web:
#     pickle.dump(data_box, web)
# print('saved ', finename)

# finename = save_pkl_dir+'/'+exp_code+'.pkl'
# with open(finename, 'rb') as web:
#       data_box = pickle.load(web)

In [ ]:
# joint_name_list = [
#         ['pelvis', 0],
#         ['naval', 1],
#         ['chest', 2],
#         ['neck', 3],
#         ['l_clavicle', 4],
#         ['l_shoulder', 5],
#         ['l_elbow', 6],
#         ['l_wrist', 7],
#         ['l_hand', 8],
#         ['l_handtip', 9],
#         ['l_thumb', 10],
#         ['r_clavicle', 11],
#         ['r_shoulder', 12],
#         ['r_elbow', 13],
#         ['r_wrist', 14],
#         ['r_hand', 15],
#         ['r_handtip', 16],
#         ['r_thumb', 17],
#         ['l_hip', 18],
#         ['l_knee', 19],
#         ['l_ankle', 20],
#         ['l_foot', 21],
#         ['r_hip', 22],
#         ['r_knee', 23],
#         ['r_ankle', 24],
#         ['r_foot', 25],
#         ['head', 26],
#         ['nose', 27],
#         ['l_eyes', 28],
#         ['l_ear', 29],
#         ['r_eyes', 30],
#         ['r_ear', 31],]

In [ ]:
# データ読み込み
exp_code = "00_20201012_exp"
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)

In [ ]:
# 切り出し時刻チェック
exp_code = "01_20201012_exp"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

title=exp_code+'_marge_raw_look_x'

xmin=0
xmax=4500

plt.figure(figsize=(20,4))
ic=0
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    col='data_fast_status_look_points_look_point_center_x'
    x=df[col]
    
    plt.plot(x,label=pp+ss+tt+'_'+str(is_mask),color=cmap[ic])
    ic=ic+1
#     col='data_fast_status_look_points_look_point_center_y'
#     x2=df[col]
#     plt.plot(x2,label=col,color='G',alpha=0.6)
    
plt.title(title)
plt.ylim(1200,-1200)
plt.xlim(xmin,xmax)
plt.legend(loc="upper left",frameon=False)
# save as png
save_file=title+'.png'
save_path=os.path.join(save_dir, save_file)
plt.savefig(save_path)
plt.show()   

In [ ]:
# 切り出し時刻チェック
exp_code = "01_20201012_exp"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

title=exp_code+'_marge_raw_look_y'

xmin=0
xmax=4500

plt.figure(figsize=(20,4))
ic=0
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    col='data_fast_status_look_points_look_point_center_y'
    x=df[col]
#     if(pp=='p20' and ss=='s02'):
#         x=list(df[col][300:])
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p20' and ss=='s03'):
#         x=list(df[col][300:])
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p04' and ss=='s02' and is_mask==0):
#         x=list(df[col][300:])
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p04' and ss=='s02' and is_mask==1):
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p04' and ss=='s03' and is_mask==0):
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p04' and ss=='s03' and is_mask==1):
#         print(pp,ss,is_mask,len(x))
#         #continue
    
    plt.plot(x,label=pp+ss+tt+'_'+str(is_mask),color=cmap[ic])
    ic=ic+1
#     col='data_fast_status_look_points_look_point_center_y'
#     x2=df[col]
#     plt.plot(x2,label=col,color='G',alpha=0.6)
    
plt.title(title)
plt.ylim(1200,-1200)
plt.xlim(xmin,xmax)
plt.legend(loc="upper left",frameon=False)
# save as png
save_file=title+'.png'
save_path=os.path.join(save_dir, save_file)
plt.savefig(save_path)
plt.show() 

In [ ]:
# 試行時間割 全部　調整
# 時系列チェック
exp_code = "01_20201015_exp_fix"
save_dir = 'img/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

st_box=[]
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    # 時刻
#     if not (pp=='p20' and ss=='s02' and is_mask==1):
#         continue
    if(pp=='p04' and ss=='s03' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 535, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            
            [1190,1250, 5],[1300,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
           
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2230,2245, 3],[2275,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2520,2545, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2930, 3],[2960,2990, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit
        #continue
    elif(pp=='p04' and ss=='s03' and is_mask==1):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2910,2940, 3],[2965,2995, 2],[3030,3060, 1],[3080,3120, 4],
            [3140,3180, 7],[3200,3240, 8],[3270,3295, 9],[3340,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3690, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 850, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2920, 3],[2960,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==1):#
        front_box=[ 
             [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 540, 600, 3],[ 630, 700, 9],[ 765, 805, 7],[ 860, 900, 1],[ 950,1000, 5],
            [1200,1250, 5],[1290,1350, 3],[1390,1450, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2035,2050, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2480, 7],[2510,2550, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2860, 5],[2910,2940, 3],[2970,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3265,3280, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3660,3690, 5],[3865,3900, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit
        #continue
    else:
        # 時刻
        front_box=[
            [  0,  300, 0],
                   [ 300, 400, 5],
                   [ 400, 500, 1],
                   [ 500, 600, 3],
                   [ 600, 700, 9],
                   [ 700, 800, 7],
                   [ 800, 900, 1],
                   [ 900,1000, 5],
                   
                   [1150,1250, 5],
                   [1250,1350, 3],
                   [1350,1450, 1],
                   [1450,1550, 7],
                   [1550,1650, 9],
                   [1650,1750, 3],
                   [1750,1850, 5],
                   
                   [2000,2060, 5],
                   [2060,2120, 1],
                   [2120,2180, 2],
                   [2180,2240, 3],
                   [2240,2300, 6],
                   [2300,2360, 9],
                   [2360,2420, 8],
                   [2420,2480, 7],
                   [2480,2540, 4],
                   [2540,2600, 1],
                   [2600,2660, 5],
                   
                   [2810,2870, 5],
                   [2870,2930, 1],
                   [2930,2990, 2],
                   [2990,3050, 3],
                   [3050,3110, 6],
                   [3110,3170, 9],
                   [3170,3230, 8],
                   [3230,3290, 7],
                   [3290,3350, 4],
                   [3350,3410, 1],
                   [3410,3470, 5],
                   
                   [3620,3680, 5],
                   [3830,3890, 5],
                   [4040,4100, 5],
                   [4250,4310, 5],
                   [4310,4460, 0],
                   [4460,4500, 0],
                  ]
        continue

    # 計算
    col='data_fast_status_look_points_look_point_center_x'
    x3_1=df[col].rolling(10).max()
    x3_2=df[col].rolling(10).min()
    dx3=np.abs(x3_1-x3_2)
    col='data_fast_status_look_points_look_point_center_y'
    y3_1=df[col].rolling(10).max()
    y3_2=df[col].rolling(10).min()
    dy3=np.abs(y3_1-y3_2)
    z=dx3+dy3
    df['dz']=z
    df['moving']=0
    df['stable']=0
    mvth=200
    for i in range(len(df)):
        dz=df.at[i,'dz']
        if(dz>mvth):
            df.at[i,'moving']=1
            df.at[i,'stable']=0
        else:
            df.at[i,'moving']=0
            df.at[i,'stable']=1
    df['auto_flag']=0
    for box in front_box:
        stp=box[0]
        fnp=box[1]
        for i in range(stp,fnp):
            stableflag=df.at[i,'stable']
            if(stableflag==1):
                df.at[i,'auto_flag']=box[2]

    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_look_xy_mask_'+str(is_mask)
    def plot_look_point(title,cmap,front_box,save_dir):        
        plt.figure(figsize=(12,4))
        col='data_status_look_points_look_point_center_x'
        x=df[col]
        plt.plot(x,label=col,color='R',alpha=0.4)
        col='data_fast_status_look_points_look_point_center_x'
        x=df[col]
        plt.plot(x,label=col,color='R',alpha=0.6)
        col='data_status_look_points_look_point_center_y'
        x2=df[col]
        plt.plot(x2,label=col,color='G',alpha=0.4)
        col='data_fast_status_look_points_look_point_center_y'
        x2=df[col]
        plt.plot(x2,label=col,color='G',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        plt.ylim(1200,-1200)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_look_point(title,cmap,front_box,save_dir)
    
    #title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_dd_mask_'+str(is_mask)
    def plot_look_point_d(title,cmap,front_box,save_dir):
        plt.figure(figsize=(12,4))
        z=df['dz']
        plt.plot(z,label=col,color='B',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        plt.ylim(0,1000)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    #plot_look_point_d(title,cmap,front_box,save_dir)   
    
    def plot_look_point_d(title,cmap,front_box,save_dir):
        plt.figure(figsize=(12,4))
        z=df['stable']
        plt.plot(z,label=col,color='B',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        plt.ylim(0,1.5)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    #title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_stable_flag_mask_'+str(is_mask) 
    #plot_look_point_d(title,cmap,front_box,save_dir)        

    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_auto_flag_mask_'+str(is_mask) 
    def plot_look_point_d(title,cmap,front_box,save_dir):
        plt.figure(figsize=(12,4))
        z=df['auto_flag']
        plt.plot(z,label=col,color='B',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        plt.ylim(0,12)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()

    plot_look_point_d(title,cmap,front_box,save_dir)     
    

In [ ]:
# 試行時間割 全部　調整用
# 時系列チェック
exp_code = "01_20201012_exp"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

st_box=[]
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=int(box[6])
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    print(pp,ss,tt,is_mask)
    # 時刻
#     if not (pp=='p20' and ss=='s02' and is_mask==1):
#         continue
    if(pp=='p04' and ss=='s03' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 535, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1300,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2230,2245, 3],[2275,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2520,2545, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2930, 3],[2960,2990, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        continue
    elif(pp=='p04' and ss=='s03' and is_mask==1):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2910,2940, 3],[2965,2995, 2],[3030,3060, 1],[3080,3120, 4],
            [3140,3180, 7],[3200,3240, 8],[3270,3295, 9],[3340,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3690, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        continue
    elif(pp=='p04' and ss=='s02' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 850, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2920, 3],[2960,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        continue
    elif(pp=='p04' and ss=='s02' and is_mask==1):#
        front_box=[ 
             [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 540, 600, 3],[ 630, 700, 9],[ 765, 805, 7],[ 860, 900, 1],[ 950,1000, 5],
            [1200,1250, 5],[1290,1350, 3],[1390,1450, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2035,2050, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2480, 7],[2510,2550, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2860, 5],[2910,2940, 3],[2970,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3265,3280, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3660,3690, 5],[3865,3900, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit
        #continue
    else:
        # 時刻
        front_box=[
            [  0,  300, 0],
                   [ 300, 400, 5],
                   [ 400, 500, 1],
                   [ 500, 600, 3],
                   [ 600, 700, 9],
                   [ 700, 800, 7],
                   [ 800, 900, 1],
                   [ 900,1000, 5],
                   
                   [1150,1250, 5],
                   [1250,1350, 3],
                   [1350,1450, 1],
                   [1450,1550, 7],
                   [1550,1650, 9],
                   [1650,1750, 3],
                   [1750,1850, 5],
                   
                   [2000,2060, 5],
                   [2060,2120, 1],
                   [2120,2180, 2],
                   [2180,2240, 3],
                   [2240,2300, 6],
                   [2300,2360, 9],
                   [2360,2420, 8],
                   [2420,2480, 7],
                   [2480,2540, 4],
                   [2540,2600, 1],
                   [2600,2660, 5],
                   
                   [2810,2870, 5],
                   [2870,2930, 1],
                   [2930,2990, 2],
                   [2990,3050, 3],
                   [3050,3110, 6],
                   [3110,3170, 9],
                   [3170,3230, 8],
                   [3230,3290, 7],
                   [3290,3350, 4],
                   [3350,3410, 1],
                   [3410,3470, 5],
                   
                   [3620,3680, 5],
                   [3830,3890, 5],
                   [4040,4100, 5],
                   [4250,4310, 5],
                   [4310,4460, 0],
                   [4460,4500, 0],
                  ]
        continue

    print('start')
    # 計算
    col='data_fast_status_look_points_look_point_center_x'
    x3_1=df[col].rolling(10).max()
    x3_2=df[col].rolling(10).min()
    dx3=np.abs(x3_1-x3_2)
    col='data_fast_status_look_points_look_point_center_y'
    y3_1=df[col].rolling(10).max()
    y3_2=df[col].rolling(10).min()
    dy3=np.abs(y3_1-y3_2)
    z=dx3+dy3
    df['dz']=z
    df['moving']=0
    df['stable']=0
    mvth=200
    for i in range(len(df)):
        dz=df.at[i,'dz']
        if(dz>mvth):
            df.at[i,'moving']=1
            df.at[i,'stable']=0
        else:
            df.at[i,'moving']=0
            df.at[i,'stable']=1
    df['auto_flag']=0
    for box in front_box:
        stp=box[0]
        fnp=box[1]
        for i in range(stp,fnp):
            stableflag=df.at[i,'stable']
            if(stableflag==1):
                df.at[i,'auto_flag']=box[2]

    xmin=0
    xmax=4500
    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_look_xy_mask_'+str(is_mask)
    def plot_look_point(title,cmap,front_box,save_dir):        
        plt.figure(figsize=(12,4))
        col='data_status_look_points_look_point_center_x'
        x=df[col]
        plt.plot(x,label=col,color='R',alpha=0.4)
        col='data_fast_status_look_points_look_point_center_x'
        x=df[col]
        plt.plot(x,label=col,color='R',alpha=0.6)
        col='data_status_look_points_look_point_center_y'
        x2=df[col]
        plt.plot(x2,label=col,color='G',alpha=0.4)
        col='data_fast_status_look_points_look_point_center_y'
        x2=df[col]
        plt.plot(x2,label=col,color='G',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        plt.ylim(1200,-1200)
        plt.xlim(xmin,xmax)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_look_point(title,cmap,front_box,save_dir)   

    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_auto_flag_mask_'+str(is_mask) 
    def plot_look_point_d(title,cmap,front_box,save_dir):
        plt.figure(figsize=(12,4))
        z=df['auto_flag']
        plt.plot(z,label=col,color='B',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        plt.ylim(0,12)
        plt.xlim(xmin,xmax)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_look_point_d(title,cmap,front_box,save_dir)     

In [ ]:
# 試行時間割 全部　書き込み
# 時系列チェック
exp_code = "01_20201012_exp"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

st_box=[]
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    # 時刻
#     if not (pp=='p21' and ss=='s03' and is_mask==0):
#         continue   
    if(pp=='p04' and ss=='s03' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 535, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1300,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2230,2245, 3],[2275,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2520,2545, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2930, 3],[2960,2990, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    elif(pp=='p04' and ss=='s03' and is_mask==1):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2910,2940, 3],[2965,2995, 2],[3030,3060, 1],[3080,3120, 4],
            [3140,3180, 7],[3200,3240, 8],[3270,3295, 9],[3340,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3690, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 850, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2920, 3],[2960,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==1):#
        front_box=[ 
             [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 540, 600, 3],[ 630, 700, 9],[ 765, 805, 7],[ 860, 900, 1],[ 950,1000, 5],
            [1200,1250, 5],[1290,1350, 3],[1390,1450, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2035,2050, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2480, 7],[2510,2550, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2860, 5],[2910,2940, 3],[2970,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3265,3280, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3660,3690, 5],[3865,3900, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    else:
        # 時刻
        front_box=[
            [  0,  300, 0],
                   [ 300, 400, 5],
                   [ 400, 500, 1],
                   [ 500, 600, 3],
                   [ 600, 700, 9],
                   [ 700, 800, 7],
                   [ 800, 900, 1],
                   [ 900,1000, 5],
                   
                   [1150,1250, 5],
                   [1250,1350, 3],
                   [1350,1450, 1],
                   [1450,1550, 7],
                   [1550,1650, 9],
                   [1650,1750, 3],
                   [1750,1850, 5],
                   
                   [2000,2060, 5],
                   [2060,2120, 1],
                   [2120,2180, 2],
                   [2180,2240, 3],
                   [2240,2300, 6],
                   [2300,2360, 9],
                   [2360,2420, 8],
                   [2420,2480, 7],
                   [2480,2540, 4],
                   [2540,2600, 1],
                   [2600,2660, 5],
                   
                   [2810,2870, 5],
                   [2870,2930, 1],
                   [2930,2990, 2],
                   [2990,3050, 3],
                   [3050,3110, 6],
                   [3110,3170, 9],
                   [3170,3230, 8],
                   [3230,3290, 7],
                   [3290,3350, 4],
                   [3350,3410, 1],
                   [3410,3470, 5],
                   
                   [3620,3680, 5],
                   [3830,3890, 5],
                   [4040,4100, 5],
                   [4250,4310, 5],
                   [4310,4460, 0],
                   [4460,4500, 0],
                  ]
        continue
    # 注視点
    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_look_xy_mask_'+str(is_mask)
    def plot_look_point(title,cmap,front_box,save_dir):        
        plt.figure(figsize=(12,4))
        col='data_status_look_points_look_point_center_x'
        x=df[col]
        plt.plot(x,label=col,color='R',alpha=0.4)
        col='data_fast_status_look_points_look_point_center_x'
        x=df[col]
        plt.plot(x,label=col,color='R',alpha=0.6)
        col='data_status_look_points_look_point_center_y'
        x2=df[col]
        plt.plot(x2,label=col,color='G',alpha=0.4)
        col='data_fast_status_look_points_look_point_center_y'
        x2=df[col]
        plt.plot(x2,label=col,color='G',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        plt.ylim(1200,-1200)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_look_point(title,cmap,front_box,save_dir)
    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_look_dx_mask_'+str(is_mask) 
    def plot_look_point_d(title,cmap,front_box,save_dir):        
        plt.figure(figsize=(12,4))
        col='data_fast_status_look_points_look_point_center_x'
        x3_1=df[col].rolling(10).max()
        x3_2=df[col].rolling(10).min()
        dx3=np.abs(x3_1-x3_2)
        col='data_fast_status_look_points_look_point_center_y'
        y3_1=df[col].rolling(10).max()
        y3_2=df[col].rolling(10).min()
        dy3=np.abs(y3_1-y3_2)
        z=dx3+dy3
        df['dz']=z
        plt.plot(z,label=col,color='B',alpha=0.6)
        df['limit']=120
        z=df['limit']
        plt.plot(z,label=col,color='K',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        plt.ylim(0,1000)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_look_point_d(title,cmap,front_box,save_dir)
    
    # gaze flag    
    df['gaze_flag']=(-1)
    for pbox in front_box:
        for ii in range(pbox[0], pbox[1]):
            df.at[ii,'gaze_flag']=pbox[2]    
    df_c=df.copy()    
    sbox=[box[0],box[1],box[2],box[3],box[4],df_c,box[6]]
    st_box.append(sbox)
    
    #save_dir2=save_csv_dir+'/data_df_fix2'
    #if not os.path.isdir(save_dir2):
    #    os.makedirs(save_dir2)
    # 
    #filecode=save_dir2+'/part_'+pp+ss+tt+'_'+str(is_mask)+".csv"
    #df_c.to_csv(filecode,index=False)
    #print(filecode)
data_box=st_box

exp_code = exp_code+"_fix"
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'wb') as web:
    pickle.dump(data_box, web)
print('saved ', finename)

## Stable  

In [ ]:
# 試行時間割 stable　調整
# データ読み込み
exp_code = "01_20201012_exp_fix"
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)
      #print techacademy

# 時系列チェック
exp_code = "01_20201012_exp_fix2"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

st_box=[]
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    # 時刻
#     if not (pp=='p28' and ss=='s02' and is_mask==1):
#         continue   
    if(pp=='p04' and ss=='s03' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 535, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1300,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2230,2245, 3],[2275,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2520,2545, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2930, 3],[2960,2990, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    elif(pp=='p04' and ss=='s03' and is_mask==1):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2910,2940, 3],[2965,2995, 2],[3030,3060, 1],[3080,3120, 4],
            [3140,3180, 7],[3200,3240, 8],[3270,3295, 9],[3340,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3690, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 850, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2920, 3],[2960,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==1):#
        front_box=[ 
             [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 540, 600, 3],[ 630, 700, 9],[ 765, 805, 7],[ 860, 900, 1],[ 950,1000, 5],
            [1200,1250, 5],[1290,1350, 3],[1390,1450, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2035,2050, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2480, 7],[2510,2550, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2860, 5],[2910,2940, 3],[2970,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3265,3280, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3660,3690, 5],[3865,3900, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit
        #continue
    else:
        # 時刻
        front_box=[
            [  0,  300, 0],
                   [ 300, 400, 5],
                   [ 400, 500, 1],
                   [ 500, 600, 3],
                   [ 600, 700, 9],
                   [ 700, 800, 7],
                   [ 800, 900, 1],
                   [ 900,1000, 5],
                   
                   [1150,1250, 5],
                   [1250,1350, 3],
                   [1350,1450, 1],
                   [1450,1550, 7],
                   [1550,1650, 9],
                   [1650,1750, 3],
                   [1750,1850, 5],
                   
                   [2000,2060, 5],
                   [2060,2120, 1],
                   [2120,2180, 2],
                   [2180,2240, 3],
                   [2240,2300, 6],
                   [2300,2360, 9],
                   [2360,2420, 8],
                   [2420,2480, 7],
                   [2480,2540, 4],
                   [2540,2600, 1],
                   [2600,2660, 5],
                   
                   [2810,2870, 5],
                   [2870,2930, 1],
                   [2930,2990, 2],
                   [2990,3050, 3],
                   [3050,3110, 6],
                   [3110,3170, 9],
                   [3170,3230, 8],
                   [3230,3290, 7],
                   [3290,3350, 4],
                   [3350,3410, 1],
                   [3410,3470, 5],
                   
                   [3620,3680, 5],
                   [3830,3890, 5],
                   [4040,4100, 5],
                   [4250,4310, 5],
                   [4310,4460, 0],
                   [4460,4500, 0],
                  ]
        continue

    # 計算
    col='data_fast_status_look_points_look_point_center_x'
    x3_1=df[col].rolling(10).max()
    x3_2=df[col].rolling(10).min()
    dx3=np.abs(x3_1-x3_2)
    col='data_fast_status_look_points_look_point_center_y'
    y3_1=df[col].rolling(10).max()
    y3_2=df[col].rolling(10).min()
    dy3=np.abs(y3_1-y3_2)
    z=dx3+dy3
    df['dz']=z
    df['moving']=0
    df['stable']=0
    mvth=180
    for i in range(len(df)):
        dz=df.at[i,'dz']
        if(dz>mvth):
            df.at[i,'moving']=1
            df.at[i,'stable']=0
        else:
            df.at[i,'moving']=0
            df.at[i,'stable']=1
    df['auto_flag']=0
    for box in front_box:
        stp=box[0]
        fnp=box[1]
        for i in range(stp,fnp):
            stableflag=df.at[i,'stable']
            if(stableflag==1):
                df.at[i,'auto_flag']=box[2]    
    df['stable_gaze_flag']=df['auto_flag']
    # 注視点    
#     xmin=0
#     xmax=6000
    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_look_xy_mask_'+str(is_mask)
    def plot_look_point(title,cmap,front_box,save_dir):        
        plt.figure(figsize=(16,4))
        col='data_status_look_points_look_point_center_x'
        x=df[col]
        plt.plot(x,label=col,color='R',alpha=0.4)
        col='data_status_look_points_look_point_center_y'
        x2=df[col]
        plt.plot(x2,label=col,color='G',alpha=0.4)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        #plt.xlim(xmin,xmax)
        plt.ylim(1200,-1200)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_look_point(title,cmap,front_box,save_dir)
    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_look_dx_mask_'+str(is_mask) 
    def plot_look_point_d(title,cmap,front_box,save_dir):        
        plt.figure(figsize=(16,4))
        z=df['dz']
        plt.plot(z,label=col,color='B',alpha=0.6)
        df['limit']=120
        z=df['limit']
        plt.plot(z,label=col,color='K',alpha=0.6)
        f=df['gaze_flag'] * 100
        plt.plot(f,label=col,color='G',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        f2=df['auto_flag'] * 90
        plt.plot(f2,label=col,color='G',alpha=0.8)
        
        plt.title(title)
        plt.ylim(0,1000)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_look_point_d(title,cmap,front_box,save_dir)

In [ ]:
# 試行時間割 stable　書き込み
# データ読み込み
exp_code = "01_20201012_exp_fix"
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)
      #print techacademy

# 時系列チェック
exp_code = "01_20201012_exp_fix3"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

st_box=[]
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    # 時刻
#     if not (pp=='p28' and ss=='s02' and is_mask==1):
#         continue   
    if(pp=='p04' and ss=='s03' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 535, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1300,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2230,2245, 3],[2275,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2520,2545, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2930, 3],[2960,2990, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    elif(pp=='p04' and ss=='s03' and is_mask==1):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2910,2940, 3],[2965,2995, 2],[3030,3060, 1],[3080,3120, 4],
            [3140,3180, 7],[3200,3240, 8],[3270,3295, 9],[3340,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3690, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 850, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2920, 3],[2960,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==1):#
        front_box=[ 
             [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 540, 600, 3],[ 630, 700, 9],[ 765, 805, 7],[ 860, 900, 1],[ 950,1000, 5],
            [1200,1250, 5],[1290,1350, 3],[1390,1450, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2035,2050, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2480, 7],[2510,2550, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2860, 5],[2910,2940, 3],[2970,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3265,3280, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3660,3690, 5],[3865,3900, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    else:
        # 時刻
        front_box=[
            [  0,  300, 0],
                   [ 300, 400, 5],
                   [ 400, 500, 1],
                   [ 500, 600, 3],
                   [ 600, 700, 9],
                   [ 700, 800, 7],
                   [ 800, 900, 1],
                   [ 900,1000, 5],
                   
                   [1150,1250, 5],
                   [1250,1350, 3],
                   [1350,1450, 1],
                   [1450,1550, 7],
                   [1550,1650, 9],
                   [1650,1750, 3],
                   [1750,1850, 5],
                   
                   [2000,2060, 5],
                   [2060,2120, 1],
                   [2120,2180, 2],
                   [2180,2240, 3],
                   [2240,2300, 6],
                   [2300,2360, 9],
                   [2360,2420, 8],
                   [2420,2480, 7],
                   [2480,2540, 4],
                   [2540,2600, 1],
                   [2600,2660, 5],
                   
                   [2810,2870, 5],
                   [2870,2930, 1],
                   [2930,2990, 2],
                   [2990,3050, 3],
                   [3050,3110, 6],
                   [3110,3170, 9],
                   [3170,3230, 8],
                   [3230,3290, 7],
                   [3290,3350, 4],
                   [3350,3410, 1],
                   [3410,3470, 5],
                   
                   [3620,3680, 5],
                   [3830,3890, 5],
                   [4040,4100, 5],
                   [4250,4310, 5],
                   [4310,4460, 0],
                   [4460,4500, 0],
                  ]
        continue
    # 計算
    col='data_fast_status_look_points_look_point_center_x'
    x3_1=df[col].rolling(10).max()
    x3_2=df[col].rolling(10).min()
    dx3=np.abs(x3_1-x3_2)
    col='data_fast_status_look_points_look_point_center_y'
    y3_1=df[col].rolling(10).max()
    y3_2=df[col].rolling(10).min()
    dy3=np.abs(y3_1-y3_2)
    z=dx3+dy3
    df['dz']=z
    df['moving']=0
    df['stable']=0
    mvth=180
    for i in range(len(df)):
        dz=df.at[i,'dz']
        if(dz>mvth):
            df.at[i,'moving']=1
            df.at[i,'stable']=0
        else:
            df.at[i,'moving']=0
            df.at[i,'stable']=1
    df['auto_flag']=-1
    for pbox in front_box:
        stp=pbox[0]
        fnp=pbox[1]
        for i in range(stp,fnp):
            stableflag=df.at[i,'stable']
            if(stableflag==1):
                df.at[i,'auto_flag']=pbox[2]    
    df['stable_gaze_flag']=df['auto_flag']

    # 注視点
#     xmin=0
#     xmax=6000
    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_look_xy_mask_'+str(is_mask)
    def plot_look_point(title,cmap,front_box,save_dir):        
        plt.figure(figsize=(16,4))
        col='data_status_look_points_look_point_center_x'
        x=df[col]
        plt.plot(x,label=col,color='R',alpha=0.4)
        col='data_fast_status_look_points_look_point_center_x'
        x=df[col]
        plt.plot(x,label=col,color='R',alpha=0.6)
        col='data_status_look_points_look_point_center_y'
        x2=df[col]
        plt.plot(x2,label=col,color='G',alpha=0.4)
        col='data_fast_status_look_points_look_point_center_y'
        x2=df[col]
        plt.plot(x2,label=col,color='G',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        #plt.xlim(xmin,xmax)
        plt.ylim(1200,-1200)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_look_point(title,cmap,front_box,save_dir)
    title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_look_dx_mask_'+str(is_mask) 
    def plot_look_point_d(title,cmap,front_box,save_dir):        
        plt.figure(figsize=(16,4))
        z=df['dz']
        plt.plot(z,label=col,color='B',alpha=0.6)
        df['limit']=mvth
        z=df['limit']
        plt.plot(z,label=col,color='K',alpha=0.6)
        f=df['stable_gaze_flag'] * 100
        plt.plot(f,label=col,color='G',alpha=0.6)
        for box in front_box:
            plt.axvspan(box[0], box[1], color = cmap[box[2]],alpha=0.4)
        plt.title(title)
        plt.ylim(0,1000)
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_look_point_d(title,cmap,front_box,save_dir)

    # stable gaze flag 
    #print(box)
    df_c=df.copy()
    #df_c=df_c.drop(['dz','limit','auto_flag','moving','stable'], axis=1)
    sbox=[box[0],box[1],box[2],box[3],box[4],df_c,box[6]]
    st_box.append(sbox)
    
    #save_dir2=save_csv_dir+'/data_df_fix5'
    save_dir2=save_csv_dir+'/data_df_fix3'
    if not os.path.isdir(save_dir2):
        os.makedirs(save_dir2)
    # 
    filecode=save_dir2+'/part_'+pp+ss+tt+'_'+str(is_mask)+".csv"
    df_c.to_csv(filecode, index=False)
    print(filecode)
data_box=st_box
# 保存
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'wb') as web:
    pickle.dump(data_box, web)
print('saved ', finename)

In [ ]:
# データ読み込み
exp_code = "01_20201012_exp_fix3"
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)
      #print techacademy